# Исследование надежности заемщиков.

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.
Результаты исследования будут учтены при построении модели кредитного скоринга — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.


### Шаг 1. Обзор данных

In [1]:
import pandas as pd
data = pd.read_csv('/datasets/data.csv')
data.head(15) #посмотрим первые 15 строк.

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


In [2]:
data.info() #ознакомимся с общей информацией о датафрейме.

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


**Краткий обзор данных:**

А. В датафрейме 11 столбцов, названия столбцов понятны, измененний не требуют.  

Описание данных:  
`children` — количество детей в семье  
`days_employed` — общий трудовой стаж в днях  
`dob_years` — возраст клиента в годах  
`education` — уровень образования клиента  
`education_id` — идентификатор уровня образования  
`family_status` — семейное положение  
`family_status_id` — идентификатор семейного положения  
`gender` — пол клиента  
`income_type` — тип занятости  
`debt` — имел ли задолженность по возврату кредитов  
`total_income` — ежемесячный доход  
`purpose` — цель получения кредита  

Б. Типы данных в столбцах соответствуют смыслу данных. Вместе с тем, для удобства можно заменить тип данных в столбце `total_income` на целочисленный.

В данных присутствуют следующие аномалии:
1. Пропуски в столбцах `days_employed` и `total_income`, одинаковое количество.
2. В столбце `days_employed` встречаются как положительные, так и отрицательные значения, что, возможно, противоречит смыслу данных в столбце и нуждается в проверке.  
3. Видны одни и те же значения в столбце `education` в разных регистрах, что может свидетельствовать о наличии неявных дубликатов.

### Шаг 2.1 Заполнение пропусков

In [3]:
#определим количество пропущенных значений в столбцах с пропусками.
days_employed_nan = data['days_employed'].isna().sum()
total_income_nan = data['total_income'].isna().sum()
print(days_employed_nan)
total_income_nan

2174


2174

In [4]:
#определим долю пропущенных значений в столбцах с пропусками.
print(days_employed_nan / data['days_employed'].count())
total_income_nan / data['total_income'].count()

0.11234561521368405


0.11234561521368405

In [5]:
#пропущенных значений в столбцах `days_employed` и `total_income` совпадают, что может быть связано с тем,
#что пропуски в указанных столбцах допущены в одних и тех же строках. Проверим:
data[data['days_employed'].isna()].head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,NaN,65,среднее,1,гражданский брак,1,M,пенсионер,0,NaN,сыграть свадьбу
26,0,NaN,41,среднее,1,женат / замужем,0,M,госслужащий,0,NaN,образование
29,0,NaN,63,среднее,1,Не женат / не замужем,4,F,пенсионер,0,NaN,строительство жилой недвижимости
41,0,NaN,50,среднее,1,женат / замужем,0,F,госслужащий,0,NaN,сделка с подержанным автомобилем
55,0,NaN,54,среднее,1,гражданский брак,1,F,пенсионер,1,NaN,сыграть свадьбу


Количество (2174 шт.) и доля (11,2%) пропущенных значений в столбцах `days_employed` и `total_income` совпадают, пропуски в указанных столбцах допущены в одних и тех же строках.  
Это может быть связано с технологическими ошибками при выгрузки данных. Например, из одного и того же источника не были по каким-то причинам выгружены данные о доходе и количестве дней работы.  
Вышеуказанные данные являются количественными и могут принимать широкий диапазон значений, таким образом пропуски в этих данных оптимально заполнить медианными значениями из соответствующих столбцов. Вместе с тем, для столбца `days_employed` сначала небходимо решить вопрос с отрицательными и положительными значениями.

Заполним пропуски в столбце `total_income` медианным значением по столбцу.

In [6]:
data['total_income'].median() #посмотрим медианное значение

145017.93753253992

In [7]:
data['total_income'] = data['total_income'].fillna(data['total_income'].median()) #заменим пропуски на медиану
data.info() #проверим

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      21525 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


### Шаг 2.2 Проверка данных на аномалии и исправления.

#### Шаг 2.2.1 Дни трудового стажа.

In [9]:
data.sort_values(by='days_employed').head() #отсортируем по возрастанию данные по столбцу days_employed

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
16335,1,-18388.949901,61,среднее,1,женат / замужем,0,F,сотрудник,0,186178.934089,операции с недвижимостью
4299,0,-17615.563266,61,среднее,1,женат / замужем,0,F,компаньон,0,122560.741753,покупка жилья
7329,0,-16593.472817,60,высшее,0,женат / замужем,0,F,сотрудник,0,124697.846781,заняться высшим образованием
17838,0,-16264.699501,59,среднее,1,женат / замужем,0,F,сотрудник,0,51238.967133,на покупку автомобиля
16825,0,-16119.687737,64,среднее,1,женат / замужем,0,F,сотрудник,0,91527.685995,покупка жилой недвижимости


In [10]:
data.sort_values(by='days_employed', ascending=False).head() #отсортируем по убыванию данные по столбцу days_employed

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
6954,0,401755.400475,56,среднее,1,вдовец / вдова,2,F,пенсионер,0,176278.441171,ремонт жилью
10006,0,401715.811749,69,высшее,0,Не женат / не замужем,4,F,пенсионер,0,57390.256908,получение образования
7664,1,401675.093434,61,среднее,1,женат / замужем,0,F,пенсионер,0,126214.519212,операции с жильем
2156,0,401674.466633,60,среднее,1,женат / замужем,0,M,пенсионер,0,325395.724541,автомобили
7794,0,401663.850046,61,среднее,1,гражданский брак,1,F,пенсионер,0,48286.441362,свадьба


Количество дней трудового стажа изменяется в диапазоне от -18.4 тысячи до 401.8 тысяч дней.  
Отрицательные значения могли образоваться в результате ошибок ввода. Также заметим, что 401.8 тысяч дней, отработанных за 56 лет жизни кажется слишком большим, ведь среднее количество рабочих дней в году при 5-дневной рабочей неделе в России составляет 247 дней.  
Проверим:

In [11]:
data.describe() #изучим распределение количественного столбца другим методом 

,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
count,21525.000000,19351.000000,21525.000000,21525.000000,21525.000000,21525.000000,2.152500e+04
mean,0.538908,63046.497661,43.293380,0.817236,0.972544,0.080883,1.651595e+05
std,1.381587,140827.311974,12.574584,0.548138,1.420324,0.272661,9.786607e+04
min,-1.000000,-18388.949901,0.000000,0.000000,0.000000,0.000000,2.066726e+04
25%,0.000000,-2747.423625,33.000000,1.000000,0.000000,0.000000,1.077982e+05
50%,0.000000,-1203.369529,42.000000,1.000000,0.000000,0.000000,1.450179e+05
75%,1.000000,-291.095954,53.000000,1.000000,1.000000,0.000000,1.955436e+05
max,20.000000,401755.400475,75.000000,4.000000,4.000000,1.000000,2.265604e+06


In [12]:
#переведем дни трудового стажа в часы и разделим количество дней трудового стажа на предполагаемое количество лет,
#которое заемщик мог работать за свою жизнь с учетом возраста (и начиная с 18 лет)
data['days_employed_per_year'] = data['days_employed'] / (data['dob_years'] - 18)
data.sort_values(by='days_employed_per_year', ascending=False).head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,days_employed_per_year
1242,0,334764.259831,22,Среднее,1,Не женат / не замужем,4,F,пенсионер,0,89368.600062,получение высшего образования,83691.064958
19439,0,389397.167577,26,высшее,0,женат / замужем,0,F,пенсионер,0,214963.301941,покупка недвижимости,48674.645947
16166,0,364348.197352,26,среднее,1,гражданский брак,1,M,пенсионер,0,80044.196101,автомобиль,45543.524669
12507,0,379492.102505,27,среднее,1,Не женат / не замужем,4,F,пенсионер,0,50969.340462,сделка с автомобилем,42165.789167
13953,0,376824.585817,27,среднее,1,в разводе,3,M,пенсионер,0,97961.993557,операции с жильем,41869.398424


83.7 тысяч дней отработано за каждый год!  
Возможно эти данные записаны не в днях, а, например, в часах или минутах.  
Проверим тремя способами:  
1. Данные в столбце `days_employed` приведем к рабочим дням (разделим на 8 рабочих часов) - проверим таким образом, не в рабочих ли часах приведены данные.  
2. Данные в столбце `days_employed` приведем к рабочим дням (разделим на 24 часа) - проверим таким образом, не в часах ли приведены данные.  
3. Данные в столбце `days_employed` приведем к рабочим минутам (разделим на 8 рабочих часов и еще на 60 минут) - проверим таким образом, не в рабочих ли минутах приведены данные.

In [13]:
#в рабочих часах
data['days_employed_per_year'] = (data['days_employed'] / 8 ) / (data['dob_years'] - 18)
data.sort_values(by='days_employed_per_year', ascending=False).head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,days_employed_per_year
1242,0,334764.259831,22,Среднее,1,Не женат / не замужем,4,F,пенсионер,0,89368.600062,получение высшего образования,10461.383120
19439,0,389397.167577,26,высшее,0,женат / замужем,0,F,пенсионер,0,214963.301941,покупка недвижимости,6084.330743
16166,0,364348.197352,26,среднее,1,гражданский брак,1,M,пенсионер,0,80044.196101,автомобиль,5692.940584
12507,0,379492.102505,27,среднее,1,Не женат / не замужем,4,F,пенсионер,0,50969.340462,сделка с автомобилем,5270.723646
13953,0,376824.585817,27,среднее,1,в разводе,3,M,пенсионер,0,97961.993557,операции с жильем,5233.674803


In [14]:
#в астрономических часах
data['days_employed_per_year'] = (data['days_employed'] / 24) / (data['dob_years'] - 18)
data.sort_values(by='days_employed_per_year', ascending=False).head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,days_employed_per_year
1242,0,334764.259831,22,Среднее,1,Не женат / не замужем,4,F,пенсионер,0,89368.600062,получение высшего образования,3487.127707
19439,0,389397.167577,26,высшее,0,женат / замужем,0,F,пенсионер,0,214963.301941,покупка недвижимости,2028.110248
16166,0,364348.197352,26,среднее,1,гражданский брак,1,M,пенсионер,0,80044.196101,автомобиль,1897.646861
12507,0,379492.102505,27,среднее,1,Не женат / не замужем,4,F,пенсионер,0,50969.340462,сделка с автомобилем,1756.907882
13953,0,376824.585817,27,среднее,1,в разводе,3,M,пенсионер,0,97961.993557,операции с жильем,1744.558268


In [15]:
#в рабочих минутах
data['days_employed_per_year'] = (data['days_employed'] / 8 / 60) / (data['dob_years'] - 18)
data.sort_values(by='days_employed_per_year', ascending=False).head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,days_employed_per_year
1242,0,334764.259831,22,Среднее,1,Не женат / не замужем,4,F,пенсионер,0,89368.600062,получение высшего образования,174.356385
19439,0,389397.167577,26,высшее,0,женат / замужем,0,F,пенсионер,0,214963.301941,покупка недвижимости,101.405512
16166,0,364348.197352,26,среднее,1,гражданский брак,1,M,пенсионер,0,80044.196101,автомобиль,94.882343
12507,0,379492.102505,27,среднее,1,Не женат / не замужем,4,F,пенсионер,0,50969.340462,сделка с автомобилем,87.845394
13953,0,376824.585817,27,среднее,1,в разводе,3,M,пенсионер,0,97961.993557,операции с жильем,87.227913


Все ответы не устраивают:  
1. В рабочих часах - 10,5 тысяч дней в год - слишком много.  
2. В астрономических часах - 3,5 тысяч дней в год - слишком много.  
3. В рабочих минутах - 174 дня в год и менее - слишком мало.  

Таким образом, данные о количестве дней трудового стажа аномальны, следует уточнить у коллег.  

Вместе с тем, для поставленных задач количество дней трудового стажа не является необходимыми данными, поэтому просто заменими отрицательные значения на положительные и заполним пропуски медианными значениями. А перед этим удалим созданный для проверки столбец `days_employed_per_year`

In [16]:
data = data[['children', 'days_employed', 'dob_years', 'education', 'education_id', 'family_status', 
            'family_status_id', 'gender', 'income_type', 'debt', 'total_income', 'purpose']]
data.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


In [17]:
data['days_employed'] = abs(data['days_employed']) #приведем значения нужного столбца к модулю
data.sort_values(by='days_employed').head() #проверим наименьшие значения - чтобы были неотрицательные

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
17437,1,24.141633,31,среднее,1,женат / замужем,0,F,сотрудник,1,166952.415427,высшее образование
8336,0,24.240695,32,высшее,0,Не женат / не замужем,4,M,сотрудник,0,124115.373655,получение дополнительного образования
6157,2,30.195337,47,среднее,1,гражданский брак,1,M,компаньон,0,231461.185606,свадьба
9683,0,33.520665,43,среднее,1,Не женат / не замужем,4,M,сотрудник,1,128555.897209,приобретение автомобиля
2127,1,34.701045,31,высшее,0,женат / замужем,0,F,компаньон,0,90557.994311,получение образования


In [18]:
data['days_employed'] = data['days_employed'].fillna(data['days_employed'].median()) #заменим пропуски на медиану
data.info() #проверим

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     21525 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      21525 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


#### Шаг 2.2.2 Дети.

In [19]:
data['children'].value_counts() #проверим значения в столбце

 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64

In [20]:
data[data['children'] == 20] #посмортим строки с 20 детьми

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
606,20,880.221113,21,среднее,1,женат / замужем,0,M,компаньон,0,145334.865002,покупка жилья
720,20,855.595512,44,среднее,1,женат / замужем,0,F,компаньон,0,112998.738649,покупка недвижимости
1074,20,3310.411598,56,среднее,1,женат / замужем,0,F,сотрудник,1,229518.537004,получение образования
2510,20,2714.161249,59,высшее,0,вдовец / вдова,2,F,сотрудник,0,264474.835577,операции с коммерческой недвижимостью
2941,20,2161.591519,0,среднее,1,женат / замужем,0,F,сотрудник,0,199739.941398,на покупку автомобиля
...,...,...,...,...,...,...,...,...,...,...,...,...
21008,20,1240.257910,40,среднее,1,женат / замужем,0,F,сотрудник,1,133524.010303,свой автомобиль
21325,20,601.174883,37,среднее,1,женат / замужем,0,F,компаньон,0,102986.065978,профильное образование
21390,20,2194.220567,53,среднее,1,женат / замужем,0,M,компаньон,0,145017.937533,покупка жилой недвижимости
21404,20,494.788448,52,среднее,1,женат / замужем,0,M,компаньон,0,156629.683642,операции со своей недвижимостью


In [21]:
data[data['children'] == -1] #посмортим строки с -1 детьми

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
291,-1,4417.703588,46,среднее,1,гражданский брак,1,F,сотрудник,0,102816.346412,профильное образование
705,-1,902.084528,50,среднее,1,женат / замужем,0,F,госслужащий,0,137882.899271,приобретение автомобиля
742,-1,3174.456205,57,среднее,1,женат / замужем,0,F,сотрудник,0,64268.044444,дополнительное образование
800,-1,349987.852217,54,среднее,1,Не женат / не замужем,4,F,пенсионер,0,86293.724153,дополнительное образование
941,-1,2194.220567,57,Среднее,1,женат / замужем,0,F,пенсионер,0,145017.937533,на покупку своего автомобиля
1363,-1,1195.264956,55,СРЕДНЕЕ,1,женат / замужем,0,F,компаньон,0,69550.699692,профильное образование
1929,-1,1461.303336,38,среднее,1,Не женат / не замужем,4,M,сотрудник,0,109121.569013,покупка жилья
2073,-1,2539.761232,42,среднее,1,в разводе,3,F,компаньон,0,162638.609373,покупка жилья
3814,-1,3045.290443,26,Среднее,1,гражданский брак,1,F,госслужащий,0,131892.785435,на проведение свадьбы
4201,-1,901.101738,41,среднее,1,женат / замужем,0,F,госслужащий,0,226375.766751,операции со своей недвижимостью


Вероятно, -1 ребенок должен значить 1, а 20 детей - 2. Предположительно, это человеческий фактор - ошибки ввода данных.  
Таких значений получается 76 + 47 = 123 из 21525 значений в таблице. Поскольку достоверно сказать про количество детей нельзя в этих случаях - удалим эти данные. Они не должны существенно повлиять на результат.

In [22]:
data = data[(data['children'] != -1) & (data['children'] != 20)] #удаляем
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21402 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21402 non-null  int64  
 1   days_employed     21402 non-null  float64
 2   dob_years         21402 non-null  int64  
 3   education         21402 non-null  object 
 4   education_id      21402 non-null  int64  
 5   family_status     21402 non-null  object 
 6   family_status_id  21402 non-null  int64  
 7   gender            21402 non-null  object 
 8   income_type       21402 non-null  object 
 9   debt              21402 non-null  int64  
 10  total_income      21402 non-null  float64
 11  purpose           21402 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.1+ MB


#### Шаг 2.2.3 Возраст.

In [23]:
data['dob_years'].value_counts().sort_values() #проверим значения в столбце

75      1
74      6
73      8
19     14
72     33
20     51
71     58
70     65
69     83
68     99
0     100
21    110
67    167
22    183
66    183
65    194
23    252
64    263
24    263
63    268
62    351
61    353
25    356
60    376
26    406
55    441
59    441
51    446
57    457
53    457
58    461
46    469
54    476
47    480
56    482
52    483
27    490
45    494
28    501
49    505
32    506
50    509
43    510
37    531
30    536
48    536
44    543
29    543
36    553
31    556
39    572
33    577
42    592
38    595
34    597
41    603
40    603
35    614
Name: dob_years, dtype: int64

In [24]:
data.describe()

,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
count,21402.000000,21402.000000,21402.000000,21402.000000,21402.000000,21402.000000,2.140200e+04
mean,0.473180,60478.305230,43.300206,0.817120,0.973133,0.080927,1.651829e+05
std,0.751837,133355.117133,12.579055,0.548741,1.420536,0.272729,9.799192e+04
min,0.000000,24.141633,0.000000,0.000000,0.000000,0.000000,2.066726e+04
25%,0.000000,1027.540529,33.000000,1.000000,0.000000,0.000000,1.077163e+05
50%,0.000000,2194.220567,42.000000,1.000000,0.000000,0.000000,1.450179e+05
75%,1.000000,4792.258983,53.000000,1.000000,1.000000,0.000000,1.955726e+05
max,5.000000,401755.400475,75.000000,4.000000,4.000000,1.000000,2.265604e+06


In [25]:
data[data['dob_years'] == 0] #посмортим строки с людьми нулевого возраста

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
99,0,346541.618895,0,Среднее,1,женат / замужем,0,F,пенсионер,0,71291.522491,автомобиль
149,0,2664.273168,0,среднее,1,в разводе,3,F,сотрудник,0,70176.435951,операции с жильем
270,3,1872.663186,0,среднее,1,женат / замужем,0,F,сотрудник,0,102166.458894,ремонт жилью
578,0,397856.565013,0,среднее,1,женат / замужем,0,F,пенсионер,0,97620.687042,строительство собственной недвижимости
1040,0,1158.029561,0,высшее,0,в разводе,3,F,компаньон,0,303994.134987,свой автомобиль
...,...,...,...,...,...,...,...,...,...,...,...,...
19829,0,2194.220567,0,среднее,1,женат / замужем,0,F,сотрудник,0,145017.937533,жилье
20462,0,338734.868540,0,среднее,1,женат / замужем,0,F,пенсионер,0,259193.920299,покупка своего жилья
20577,0,331741.271455,0,среднее,1,Не женат / не замужем,4,F,пенсионер,0,129788.762899,недвижимость
21179,2,108.967042,0,высшее,0,женат / замужем,0,M,компаньон,0,240702.007382,строительство жилой недвижимости


Предположительно то, что есть 100 людей с нулевым возрастом, это человеческий фактор - ошибки ввода данных.  
Поскольку достоверно сказать про возраст нельзя в этих случаях - удалим эти данные. Они не должны существенно повлиять на результат.

In [26]:
data = data[data['dob_years'] != 0] #удаляем
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21302 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21302 non-null  int64  
 1   days_employed     21302 non-null  float64
 2   dob_years         21302 non-null  int64  
 3   education         21302 non-null  object 
 4   education_id      21302 non-null  int64  
 5   family_status     21302 non-null  object 
 6   family_status_id  21302 non-null  int64  
 7   gender            21302 non-null  object 
 8   income_type       21302 non-null  object 
 9   debt              21302 non-null  int64  
 10  total_income      21302 non-null  float64
 11  purpose           21302 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.1+ MB


#### Шаг 2.2.4 Пол.

In [27]:
data['gender'].unique() #проверяем столбец на аномалии

array(['F', 'M', 'XNA'], dtype=object)

In [28]:
data[data['gender'] == 'XNA'] #посмортим строки с людьми нулевого возраста

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
10701,0,2358.600502,24,неоконченное высшее,2,гражданский брак,1,XNA,компаньон,0,203905.157261,покупка недвижимости


Предположительно, это человеческий фактор - ошибки ввода данных, либо пол неизвестен. По данным других ячеек пол восстановить невозможно.  
Поскольку достоверно сказать про пол нельзя в этом случае - удалим эти данные. Они не должны существенно повлиять на результат.

In [29]:
data = data[data['gender'] != 'XNA'] #удаляем
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21301 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21301 non-null  int64  
 1   days_employed     21301 non-null  float64
 2   dob_years         21301 non-null  int64  
 3   education         21301 non-null  object 
 4   education_id      21301 non-null  int64  
 5   family_status     21301 non-null  object 
 6   family_status_id  21301 non-null  int64  
 7   gender            21301 non-null  object 
 8   income_type       21301 non-null  object 
 9   debt              21301 non-null  int64  
 10  total_income      21301 non-null  float64
 11  purpose           21301 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.1+ MB


#### Шаг 2.2.5 Долг.

In [30]:
data['debt'].value_counts() #проверяем столбец на аномалии

0    19577
1     1724
Name: debt, dtype: int64

Все в порядке.

#### Шаг 2.2.6 Доход.

In [31]:
data['total_income'].value_counts() #проверяем столбец на аномалии

145017.937533    2153
169846.427535       1
170140.758021       1
165432.425282       1
308173.233260       1
                 ... 
98360.185970        1
208520.564033       1
117329.195869       1
164337.653574       1
189255.286637       1
Name: total_income, Length: 19149, dtype: int64

Все в порядке. 2153 медианных значения - мы ими заполняли пропуски

### Шаг 2.3. Изменение типов данных.

Изменим тип данных в столбце `total_income` - на целочисленный.

In [32]:
data['total_income'] = data['total_income'].astype(int)
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21301 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21301 non-null  int64  
 1   days_employed     21301 non-null  float64
 2   dob_years         21301 non-null  int64  
 3   education         21301 non-null  object 
 4   education_id      21301 non-null  int64  
 5   family_status     21301 non-null  object 
 6   family_status_id  21301 non-null  int64  
 7   gender            21301 non-null  object 
 8   income_type       21301 non-null  object 
 9   debt              21301 non-null  int64  
 10  total_income      21301 non-null  int64  
 11  purpose           21301 non-null  object 
dtypes: float64(1), int64(6), object(5)
memory usage: 2.1+ MB


### Шаг 2.4. Удаление дубликатов.

In [33]:
data.duplicated().sum() #узнаем количество явных дубликатов

54

In [34]:
data['education'].value_counts() #проверим столбец на дублирование значений по смыслу

среднее                13609
высшее                  4666
СРЕДНЕЕ                  764
Среднее                  700
неоконченное высшее      662
ВЫСШЕЕ                   270
Высшее                   266
начальное                250
Неоконченное высшее       47
НЕОКОНЧЕННОЕ ВЫСШЕЕ       29
НАЧАЛЬНОЕ                 17
Начальное                 15
ученая степень             4
УЧЕНАЯ СТЕПЕНЬ             1
Ученая степень             1
Name: education, dtype: int64

In [35]:
data['family_status'].value_counts() #проверим столбец на дублирование значений по смыслу

женат / замужем          12254
гражданский брак          4138
Не женат / не замужем     2783
в разводе                 1179
вдовец / вдова             947
Name: family_status, dtype: int64

In [36]:
data['income_type'].value_counts() #проверим столбец на дублирование значений по смыслу

сотрудник          10996
компаньон           5033
пенсионер           3819
госслужащий         1447
предприниматель        2
безработный            2
в декрете              1
студент                1
Name: income_type, dtype: int64

In [37]:
data['purpose'].value_counts() #проверим столбец на дублирование значений по смыслу

свадьба                                   791
на проведение свадьбы                     768
сыграть свадьбу                           764
операции с недвижимостью                  670
покупка коммерческой недвижимости         658
покупка жилья для сдачи                   649
операции с коммерческой недвижимостью     644
операции с жильем                         642
покупка жилья для семьи                   639
жилье                                     636
покупка жилья                             635
недвижимость                              628
операции со своей недвижимостью           626
строительство собственной недвижимости    626
строительство недвижимости                620
строительство жилой недвижимости          619
покупка своего жилья                      618
покупка недвижимости                      615
ремонт жилью                              607
покупка жилой недвижимости                600
на покупку своего автомобиля              501
заняться высшим образованием      

Мы проверили дублирование значений по смыслу во всех столбцах, где дублирование значений могло привести к наличию строк-дубликатов. Дублирование значений, вероятно, произошло на этапе вноса значений из-за человеческого фактора.

Приведем значение столбцов в порядок:  
    1. `education`, `family_status` - приведем к нижнему регистру.  
    2. Прооверим, сколько дубликатов с учетом неявных.  
    3. Удалим дубликаты.

In [38]:
 #приводим к нижнему регистру
data['education'] = data['education'].str.lower()
data['family_status'] = data['family_status'].str.lower()
#проверяем
display(data['education'].value_counts())
data['family_status'].value_counts()

среднее                15073
высшее                  5202
неоконченное высшее      738
начальное                282
ученая степень             6
Name: education, dtype: int64

женат / замужем          12254
гражданский брак          4138
не женат / не замужем     2783
в разводе                 1179
вдовец / вдова             947
Name: family_status, dtype: int64

In [39]:
data.duplicated().sum() #узнаем количество явных и неявных дубликатов

71

In [40]:
data = data.drop_duplicates().reset_index(drop=True)
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21230 entries, 0 to 21229
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21230 non-null  int64  
 1   days_employed     21230 non-null  float64
 2   dob_years         21230 non-null  int64  
 3   education         21230 non-null  object 
 4   education_id      21230 non-null  int64  
 5   family_status     21230 non-null  object 
 6   family_status_id  21230 non-null  int64  
 7   gender            21230 non-null  object 
 8   income_type       21230 non-null  object 
 9   debt              21230 non-null  int64  
 10  total_income      21230 non-null  int64  
 11  purpose           21230 non-null  object 
dtypes: float64(1), int64(6), object(5)
memory usage: 1.9+ MB


### Шаг 2.5. Формирование дополнительных датафреймов словарей, декомпозиция исходного датафрейма.

In [41]:
#делим на части
data_log = data[['children', 'days_employed', 'dob_years', 'education_id', 'family_status_id', 'gender', 'income_type', 
                 'debt', 'total_income', 'purpose']]
education_dict = data[['education', 'education_id']]
education_dict = education_dict.drop_duplicates().reset_index(drop=True) #удаляем дубликаты
family_status_dict = data[['family_status', 'family_status_id']]
family_status_dict = family_status_dict.drop_duplicates().reset_index(drop=True) #удаляем дубликаты
#проверяем
display(education_dict)
display(family_status_dict)
data_log.info()

,education,education_id
0,высшее,0
1,среднее,1
2,неоконченное высшее,2
3,начальное,3
4,ученая степень,4


,family_status,family_status_id
0,женат / замужем,0
1,гражданский брак,1
2,вдовец / вдова,2
3,в разводе,3
4,не женат / не замужем,4


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21230 entries, 0 to 21229
Data columns (total 10 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21230 non-null  int64  
 1   days_employed     21230 non-null  float64
 2   dob_years         21230 non-null  int64  
 3   education_id      21230 non-null  int64  
 4   family_status_id  21230 non-null  int64  
 5   gender            21230 non-null  object 
 6   income_type       21230 non-null  object 
 7   debt              21230 non-null  int64  
 8   total_income      21230 non-null  int64  
 9   purpose           21230 non-null  object 
dtypes: float64(1), int64(6), object(3)
memory usage: 1.6+ MB


### Шаг 2.6. Категоризация дохода.

In [42]:
import warnings

warnings.filterwarnings('ignore') #избавимся от предупреждений

def total_income_category(total_income):
    """
    Возвращает группу дохода по значению дохода, используя правила:
    0–30000 — 'E';
    30001–50000 — 'D';
    50001–200000 — 'C';
    200001–1000000 — 'B';
    1000001 и выше — 'A'.
    """
    
    if total_income <= 30000:
        return 'E (0-30 т.р.)'
    if total_income <= 50000:
        return 'D (30-50 т.р.)'
    if total_income <= 200000:
        return 'C (50-200 т.р.)'
    if total_income <= 1000000:
        return 'B (200-1000 т.р.)'
    return 'A (>1000 т.р.)'
#применим функцию и создадим новый столбец с категориями дохода
data_log['total_income_category'] = data_log['total_income'].apply(total_income_category) 
data_log.head() #проверим

,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose,total_income_category
0,1,8437.673028,42,0,0,F,сотрудник,0,253875,покупка жилья,B (200-1000 т.р.)
1,1,4024.803754,36,1,0,F,сотрудник,0,112080,приобретение автомобиля,C (50-200 т.р.)
2,0,5623.422610,33,1,0,M,сотрудник,0,145885,покупка жилья,C (50-200 т.р.)
3,3,4124.747207,32,1,0,M,сотрудник,0,267628,дополнительное образование,B (200-1000 т.р.)
4,0,340266.072047,53,1,1,F,пенсионер,0,158616,сыграть свадьбу,C (50-200 т.р.)


### Шаг 2.7. Категоризация целей кредита.

In [43]:
def purpose_category(purpose):
    """
    Возвращает категорию цели кредита по значению цели кредита:
    'операции с автомобилем',
    'операции с недвижимостью',
    'проведение свадьбы',
    'получение образования'.
    """
    purpose_category_auto = ['на покупку своего автомобиля', 'заняться высшим образованием', 'автомобиль', 
                             'сделка с подержанным автомобилем', 'свой автомобиль', 'на покупку подержанного автомобиля', 
                             'автомобили', 'на покупку автомобиля', 'приобретение автомобиля', 'сделка с автомобилем']
    purpose_category_property = ['операции с недвижимостью', 'покупка коммерческой недвижимости', 'покупка жилья для сдачи', 
                                 'операции с жильем', 'операции с коммерческой недвижимостью', 'жилье', 'покупка жилья', 
                                 'покупка жилья для семьи', 'строительство собственной недвижимости', 'недвижимость', 
                                 'операции со своей недвижимостью', 'строительство жилой недвижимости', 'покупка недвижимости', 
                                 'покупка своего жилья', 'строительство недвижимости', 'ремонт жилью', 
                                 'покупка жилой недвижимости']
    purpose_category_marriage = ['свадьба', 'на проведение свадьбы', 'сыграть свадьбу']
    purpose_category_education = ['дополнительное образование', 'высшее образование', 'получение дополнительного образования', 
                                  'образование', 'получение образования', 'профильное образование', 
                                  'получение высшего образования', 'заняться образованием']
    
    if purpose in purpose_category_auto:
        return 'операции с автомобилем'
    if purpose in purpose_category_property:
        return 'операции с недвижимостью'
    if purpose in purpose_category_marriage:
        return 'проведение свадьбы'
    if purpose in purpose_category_education:
        return 'получение образования'

#применим функцию и создадим новый столбец с категориями дохода
data_log['purpose_category'] = data_log['purpose'].apply(purpose_category) 
data_log.head() #проверим

,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose,total_income_category,purpose_category
0,1,8437.673028,42,0,0,F,сотрудник,0,253875,покупка жилья,B (200-1000 т.р.),операции с недвижимостью
1,1,4024.803754,36,1,0,F,сотрудник,0,112080,приобретение автомобиля,C (50-200 т.р.),операции с автомобилем
2,0,5623.422610,33,1,0,M,сотрудник,0,145885,покупка жилья,C (50-200 т.р.),операции с недвижимостью
3,3,4124.747207,32,1,0,M,сотрудник,0,267628,дополнительное образование,B (200-1000 т.р.),получение образования
4,0,340266.072047,53,1,1,F,пенсионер,0,158616,сыграть свадьбу,C (50-200 т.р.),проведение свадьбы


### Ответы на вопросы.

Вопросы:
1. Есть ли зависимость между количеством детей и возвратом кредита в срок?
2. Есть ли зависимость между семейным положением и возвратом кредита в срок?
3. Есть ли зависимость между уровнем дохода и возвратом кредита в срок?
4. Как разные цели кредита влияют на его возврат в срок?

##### Вопрос 1: Есть ли зависимость между количеством детей и возвратом кредита в срок?

In [44]:
data_pivot = data_log.pivot_table(index = ['children'], values = 'debt', 
                                  aggfunc = ['sum', 'count'])
data_pivot['debt_percent'] = data_pivot['sum'] / data_pivot['count'] *100
data_pivot.sort_values(by='debt_percent', ascending=False)

,sum,count,debt_percent
,debt,debt,
children,,,
4,4,41,9.756098
2,194,2039,9.514468
1,441,4792,9.202838
3,27,328,8.231707
0,1058,14021,7.545824
5,0,9,0.000000


##### Вывод 1:

Количество детей у заемщиков оказывает некоторое небольшое влияние на процент заемщиков, допустивших просрочку по кредиту.  
1. В целом, чем больше детей, тем выше процент заемщиков, допустивших просрочку.  
2. Исключение составляют заемщики с количеством детей, равным трем - в этом случае процент просрочки ниже, чем у заемщиков с 1, 2, 4 детьми, но выше, чем у бездетных. Впрочем, в этой подгруппе не так много заемщиков (328 из 21230), таким образом, выьорка по подгруппе может быть нерепрезентативна.  
2. Заемщиков с 5 детьми всего 9 и ни один не допускал просрочку. По этой подгруппе выборка нерепрезентативна.  
3. Заемщиков с 4 детьми всего 41 и хотя эта подгруппа допускает больше всего просрочек, выборка может быть нерепрезентативна.

##### Вопрос 2: Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [45]:
data_pivot = data_log.pivot_table(index = 'family_status_id', values = 'debt', 
                                  aggfunc = ['sum', 'count'])
data_pivot['debt_percent'] = data_pivot['sum'] / data_pivot['count'] *100
data_pivot['family_status'] = family_status_dict['family_status']
data_pivot.sort_values(by='debt_percent', ascending=False)

,sum,count,debt_percent,family_status
,debt,debt,,
family_status_id,,,,
4,272,2780,9.784173,не женат / не замужем
1,383,4112,9.314202,гражданский брак
0,923,12213,7.557521,женат / замужем
3,84,1179,7.124682,в разводе
2,62,946,6.553911,вдовец / вдова


##### Вывод 2:

Семейное положение оказывает некоторое влияние на процент просроченных кредитов.  

1. Больше всего просрочек у заемщиков без семей, чуть меньше - у живущих в гражданском браке.  
2. Далее с существенным отрывом женатые и в разводе.  
3. Меньше всего просрочек допускают вдовы и вдовцы.

##### Вопрос 3: Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [46]:
data_pivot = data_log.pivot_table(index = ['total_income_category'], values = 'debt', 
                                  aggfunc = ['sum', 'count'])
data_pivot['debt_percent'] = data_pivot['sum'] / data_pivot['count'] *100
data_pivot.sort_values(by='debt_percent', ascending=False)

,sum,count,debt_percent
,debt,debt,
total_income_category,,,
E (0-30 т.р.),2,22,9.090909
C (50-200 т.р.),1346,15850,8.492114
A (>1000 т.р.),2,25,8.000000
B (200-1000 т.р.),353,4986,7.079824
D (30-50 т.р.),21,347,6.051873


##### Вывод 3:

Доход оказывает некоторое влияние на процент просроченных кредитов.  

1. В целом, чем выше доход заемщиков, тем меньший процент просрочек они допускают.    
2. При этом выборки по подгруппам заемщиков с доходами в диапазонах 0-30, 30-50 и  свыше 1000 тыс. рублей могут быть нерепрезентативными в силу ограниченного объема.

##### Вопрос 4: Как разные цели кредита влияют на его возврат в срок?

In [47]:
data_pivot = data_log.pivot_table(index = ['purpose_category'], values = 'debt', 
                                  aggfunc = ['sum', 'count'])
data_pivot['debt_percent'] = data_pivot['sum'] / data_pivot['count'] *100
data_pivot.sort_values(by='debt_percent', ascending=False)

,sum,count,debt_percent
,debt,debt,
purpose_category,,,
получение образования,326,3477,9.375899
операции с автомобилем,440,4751,9.261208
проведение свадьбы,181,2299,7.872988
операции с недвижимостью,777,10703,7.259647


##### Вывод 4:

Цель кредита оказывает некоторое влияние на процент просроченных кредитов.  

1. Кредиты на образование и приобретение автомобиля - наиболее рискованы.    
2. Кредиты на свадьбу менее рискованы, еще менее - кредиты на недвижимость.

## Общий вывод:

В ходе проекта изучены данные о заемщиках и просрочках исполнения ими кредитных обязательств.  

На первом этапе данные проверены на аномалии, аномалии изучены и исправлены, поскольку сделан вывод об их незначительном влиянии на результаты исследования. Пропуски в количественных данных заполнены медианными значениями, явные и скрытые дубликаты удалены. Кроме того, данные структурированы для удобства и быстроты обработки.  

На втором этапе сведения о доходах и целях кредитования категоризованы.

На третьем этапе изучено влияние количества детей, семейного положения, уровня доходов и цели кредитования на наличие просрочек исполнения обязательств по кредитам. Обобщенные выводы представлены ниже.

1. В целом, можно сделать вывод, что риск просрочки по кредиту увеличивается с возрастанием количества детей и умньшением доходов заемщиков. При этом, наиболее рисковыми подгруппами с точки зрения семейного статуса оказываются заемщики без семьи и живущие в гражданском браке. Наиболее рискованными целями кредитования являются получение образования и приобретение автомобиля.  
2. Наименее рисковые подгруппы - кредиты на приобретение недвижимости, выданные вдовцам/вдовам, заемщикам без детей и с высоким доходом.  
3. Необходима консультация с коллегами относительно данных о количестве дней трудового стажа - они не коррелируют с возрастом заемщика и количеством рабочих дней в году. Вероятно, эти данные искажены.